### Import and Install dependencies

In [1]:
import warnings;
warnings.simplefilter('ignore')

In [2]:
!pip install prophet
!pip install pystan

  Using cached pystan-3.6.0-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.3/338.3 kB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.6/338.6 kB 23.2 MB/s eta 0:00:00
  Using cached clikit-0.6.2-py2.py3-none-any.whl (91 kB)
  Using cached pystan-3.5.0-py3-none-any.whl (13 kB)
  Using cached pystan-3.4.0-py3-none-any.whl (13 kB)
  Using cached pystan-3.3.0-py3-none-any.whl (13 kB)
  Using cached pystan-3.2.0-py3-none-any.whl (13 kB)
  Using cached pystan-3.1.1-py3-none-any.whl (13 kB)
  Using cached pystan-3.1.0-py3-none-any.whl (13 kB)
  Using cached pystan-3.0.2-py3-none-any.whl (13 kB)
  Using cached pystan-3.0.1-py3-none-any.whl (12 kB)
  Using cached pystan-3.0.0-py3-none-any.whl (12 kB)
  Using cached pystan-2.19.1.1.tar.gz (16.2 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for pystan: filename=pystan-2.19.1.1-cp39-cp39-macosx_13_0_arm64.whl size=29208909 sha256=f53ccd4dafc4c36cef49155865cbdedc765751d1ae1f91

In [3]:
import pandas as pd
from prophet import Prophet

### 1. Read Data and Process Dates

In [4]:
dataset = pd.read_csv('dataset.csv')

In [5]:
dataset.describe()

,Time Date,Product,Value
count,1.080000e+03,1080.0,1080.000000
mean,1.567207e+07,2667437.0,4048.117478
std,8.791548e+06,0.0,1439.945783
min,1.012018e+06,2667437.0,2042.813500
25%,8.069518e+06,2667437.0,2632.498599
50%,1.556702e+07,2667437.0,4256.000000
75%,2.308452e+07,2667437.0,5288.248910
max,3.112202e+07,2667437.0,8147.739600


In [6]:
## checking the uniqueness of my values
print(dataset['Product'].unique())
print(dataset['Store'].unique())

[2667437]
['QLD_CW_ST0203']


In [7]:
dataset['Year']= dataset['Time Date'].apply(lambda x:str(x)[-4:])
dataset['Month']= dataset['Time Date'].apply(lambda x:str(x)[-6:-4])
dataset['Day'] = dataset['Time Date'].apply(lambda x:str(x)[:-6])
dataset ['ds'] = pd.DatetimeIndex(dataset['Year']+'-'+dataset['Month']+'-'+dataset['Day'])

In [8]:
dataset.head()

,Time Date,Product,Store,Value,Year,Month,Day,ds
0,1012018,2667437,QLD_CW_ST0203,2926.000,2018,01,1,2018-01-01
1,2012018,2667437,QLD_CW_ST0203,2687.531,2018,01,2,2018-01-02
2,3012018,2667437,QLD_CW_ST0203,2793.000,2018,01,3,2018-01-03
3,4012018,2667437,QLD_CW_ST0203,2394.000,2018,01,4,2018-01-04
4,5012018,2667437,QLD_CW_ST0203,2660.000,2018,01,5,2018-01-05


In [9]:
dataset.drop(['Time Date','Product','Store','Year','Month','Day'], axis = 1 ,inplace=True)

In [10]:
dataset.columns = ['y','ds']

In [11]:
dataset.head()

,y,ds
0,2926.000,2018-01-01
1,2687.531,2018-01-02
2,2793.000,2018-01-03
3,2394.000,2018-01-04
4,2660.000,2018-01-05


### 2. Model Training with Prophet

In [12]:
m = Prophet(interval_width= 0.95, daily_seasonality = True)
model = m.fit(dataset)

10:17:57 - cmdstanpy - INFO - Chain [1] start processing
10:17:57 - cmdstanpy - INFO - Chain [1] done processing
10:17:57 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
10:17:57 - cmdstanpy - INFO - Chain [1] start processing
10:17:57 - cmdstanpy - INFO - Chain [1] done processing
10:17:57 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/vittorio/opt/anaconda3/lib/python3.9/site-packages/prophet/stan_model/prophet_model.bin random seed=6544 data file=/var/folders/ds/rbwjh56942l0t4qk3znl00hc0000gn/T/tmpi_uhnpyt/sn9x0vxr.json init=/var/folders/ds/rbwjh56942l0t4qk3znl00hc0000gn/T/tmpi_uhnpyt/a51_1ux3.json output file=/var/folders/ds/rbwjh56942l0t4qk3znl00hc0000gn/T/tmpi_uhnpyt/prophet_model6xfovv53/prophet_model-20230321101757.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[29547]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <A9E1C977-97DD-306E-A6E0-69A5F9CD39D6> /Users/vittorio/opt/anaconda3/lib/python3.9/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/ds/rbwjh56942l0t4qk3znl00hc0000gn/T/pip-install-mjvsba5k/prophet_f42666209c824be581a4f29efdc5675f/build/lib.macosx-11.1-arm64-cpython-39/prophet/stan_model/cmdstan-2.26.1/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file)

In [ ]:
future = m.make_future_dataframe(periods= 100, freq = 'D')
forecast = m.predict (future)
forecast.head()

In [1]:
plot1 = m.plot(forecast)


NameError: name 'm' is not defined